In [2]:
import pandas as pd
import numpy as np
from datetime import datetime as dt
import os
import glob
import matplotlib.pyplot as plt
import seaborn as sns
import mpld3
import re
import math
from scipy.spatial import Voronoi, voronoi_plot_2d
from matplotlib.patches import Polygon, Patch
import matplotlib.image as mpimg
from matplotlib.ticker import ScalarFormatter
import matplotlib.dates as mdates

In [16]:
# These are the "Tableau 20" colors as RGB.    
tableau20 = [(31, 119, 180), (174, 199, 232), (255, 127, 14), (255, 187, 120),    
             (44, 160, 44), (152, 223, 138), (214, 39, 40), (255, 152, 150),    
             (148, 103, 189), (197, 176, 213), (140, 86, 75), (196, 156, 148),    
             (227, 119, 194), (247, 182, 210), (127, 127, 127), (199, 199, 199),    
             (188, 189, 34), (219, 219, 141), (23, 190, 207), (158, 218, 229)]    
  
# Scale the RGB values to the [0, 1] range, which is the format matplotlib accepts.    
for i in range(len(tableau20)):    
    r, g, b = tableau20[i]    
    tableau20[i] = (r / 255., g / 255., b / 255.)

color = tableau20

In [17]:
# allFiles = glob.glob(os.path.join("..","interpolatedData","*","*.csv"))

# df = {}
# for x in allFiles:
#     name = '_'.join(x.split(os.path.sep)[1:]).split('.')[0]
#     df[name] = pd.read_csv(x,parse_dates=[0])

# mergedDataFilesDp03 = glob.glob(os.path.join("..","..","mergedData","Dp03","*.csv"))
# mergedDataDp03 = {(route.split("Data_")[1].split(".")[0]):pd.read_csv(route,parse_dates=[0]) for route in mergedDataFilesDp03}
# mergedDataFilesPM25 = glob.glob(os.path.join("..","..","mergedData","PM25_Std","*.csv"))
# mergedDataPM25 = {(route.split("Data_")[1].split(".")[0]):pd.read_csv(route,parse_dates=[0]) for route in mergedDataFilesPM25}


# plt.rcParams.update({'font.size': 26})